<a href="https://colab.research.google.com/github/jbbonice2/PROJET-RL/blob/main/comparaison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class RowGridWorld:
    def __init__(self, n_cols=5, goal_states=[4], reward=1, step_penalty=-0.1):
        self.n_cols = n_cols
        self.goal_states = goal_states  # Liste des états cibles
        self.reward = reward
        self.step_penalty = step_penalty
        self.state = 0
        self.done = False

    def reset(self):
        self.state = np.random.randint(0, self.n_cols)  # Commencer à un état aléatoire
        self.done = False
        return self.state

    def step(self, action):
        if self.done:
            return self.state, 0, True

        if action == 0:  # Aller à gauche
            self.state = max(0, self.state - 1)
        elif action == 1:  # Aller à droite
            self.state = min(self.n_cols - 1, self.state + 1)

        # Vérifier si l'agent atteint un bord de la grille
        if self.state == 0 or self.state == self.n_cols - 1:
            self.done = True  # Fin de l'épisode
            reward = self.reward if self.state in self.goal_states else self.step_penalty
            return self.state, reward, self.done  # Fin de l'épisode avec la récompense appropriée

        # Calcul de la récompense pour chaque étape
        if self.state in self.goal_states:  # Si l'état actuel est un état cible
            reward = self.reward  # Récompense pour atteindre l'objectif
        else:
            reward = self.step_penalty  # Pénalité pour chaque mouvement effectué

        return self.state, reward, self.done

    def render(self):
        grid = ['-' for _ in range(self.n_cols)]
        grid[self.state] = 'A'
        print(" ".join(grid))

# Algorithmes
def greedy(env, n_episodes):
    rewards = []
    Q = np.zeros(2)  # Valeurs estimées pour chaque action (gauche, droite)
    N = np.zeros(2)  # Nombre de fois que chaque action a été choisie
    for _ in range(n_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = np.argmax(Q)  # Greedy: Choisir l'action avec la plus haute valeur estimée
            next_state, reward, done = env.step(action)
            N[action] += 1
            Q[action] += (reward - Q[action]) / N[action]  # Mise à jour de la valeur estimée
            total_reward += reward
        rewards.append(total_reward)
    return np.array(rewards)

def epsilon_greedy(env, n_episodes, epsilon=0.3):
    rewards = []
    Q = np.zeros(2)
    N = np.zeros(2)
    for _ in range(n_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        while not done:
            if np.random.rand() < epsilon:
                action = np.random.choice([0, 1])  # Exploration
            else:
                action = np.argmax(Q)  # Exploitation
            next_state, reward, done = env.step(action)
            N[action] += 1
            Q[action] += (reward - Q[action]) / N[action]
            total_reward += reward
        rewards.append(total_reward)
    return np.array(rewards)

def ucb(env, n_episodes, c=2):
    rewards = []
    Q = np.zeros(2)
    N = np.zeros(2)
    t = 0
    for _ in range(n_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        while not done:
            t += 1
            action = np.argmax(Q + c * np.sqrt(np.log(t + 1) / (N + 1e-5)))  # UCB
            next_state, reward, done = env.step(action)
            N[action] += 1
            Q[action] += (reward - Q[action]) / N[action]
            total_reward += reward
        rewards.append(total_reward)
    return np.array(rewards)

def gradient_bandit(env, n_episodes, alpha=0.3):
    rewards = []
    H = np.zeros(2)  # Valeurs de préférence pour chaque action
    for _ in range(n_episodes):
        state = env.reset()
        done = False
        total_reward = 0
        while not done:
            # Sélectionner une action selon les probabilités softmax
            exp = np.exp(H)
            probs = exp / np.sum(exp)
            action = np.random.choice([0, 1], p=probs)

            next_state, reward, done = env.step(action)

            # Mise à jour des préférences H
            baseline = np.mean(reward)
            H[action] += alpha * (reward - baseline)

            total_reward += reward
        rewards.append(total_reward)
    return np.array(rewards)

# Initialisation de l'environnement
env = RowGridWorld(n_cols=5, goal_states=[2, 4], reward=1, step_penalty=-0.1)
n_episodes = 100

# Comparaison des méthodes
greedy_rewards = greedy(env, n_episodes)
epsilon_greedy_rewards = epsilon_greedy(env, n_episodes)
ucb_rewards = ucb(env, n_episodes)
gradient_bandit_rewards = gradient_bandit(env, n_episodes)

# Tracer les courbes des récompenses moyennes cumulatives
plt.figure(figsize=(10, 6))

# Moyennes cumulatives
plt.plot(np.cumsum(greedy_rewards) / (np.arange(n_episodes) + 1), label='Greedy')
plt.plot(np.cumsum(epsilon_greedy_rewards) / (np.arange(n_episodes) + 1), label='Epsilon-Greedy')
plt.plot(np.cumsum(ucb_rewards) / (np.arange(n_episodes) + 1), label='UCB')
plt.plot(np.cumsum(gradient_bandit_rewards) / (np.arange(n_episodes) + 1), label='Gradient Bandit')

# Paramètres du graphique
plt.xlabel('Épisodes')
plt.ylabel('Récompense moyenne cumulative')
plt.title('Comparaison entre Greedy, Epsilon-Greedy, UCB et Gradient Bandit')
plt.legend()
plt.grid(True)
plt.show()
